In [1]:
import nltk, random
from nltk.corpus import brown, nps_chat
from pickle import dump, load

In [2]:
brown_sents = brown.sents()
brown_words = brown.words()
brown_tagged_sents = brown.tagged_sents(tagset = 'universal');

In [3]:
extra_sents = [[('Open', 'VERB'), ('Youtube', 'NOUN'), ('.', '.')], [('Close', 'VERB'), ('Youtube', 'NOUN'), ('.', '.')],
              [('Minimize', 'VERB'), ('the', 'DET'), ('window', 'NOUN'), ('.','.')],
              [('Close', 'VERB'), ('all', 'DET'), ('the', 'DET'), ('open', 'ADJ'), ('windows', 'NOUN'), ('.','.')],
              [('Shutdown', 'VERB'), ('the', 'DET'), ('system', 'NOUN'),('.','.')],
              [('shutdown', 'VERB'), ('the', 'DET'), ('system', 'NOUN'),('.','.')],
              [('shutdown', 'VERB'), ('the', 'DET'), ('tabs', 'NOUN'),('.','.')],
              [('Shutdown', 'VERB'), ('the', 'DET'), ('tabs', 'NOUN'),('.','.')],
              [('shutdown', 'VERB'), ('my', 'PRON'), ('PC', 'NOUN'),('.','.')], 
              [('Shutdown', 'VERB'), ('my', 'PRON'), ('PC', 'NOUN'),('.','.')],
              [('Open', 'VERB'), ('Gmail', 'NOUN'), ('.', '.')], [('Close', 'VERB'), ('Gmail', 'NOUN'), ('.', '.')],
              [('Open', 'VERB'), ('Email', 'NOUN'), ('.', '.')], [('Close', 'VERB'), ('Email', 'NOUN'), ('.', '.')],
              [('Login', 'VERB'), ('to', 'NOUN'), ('.', '.')], [('Close', 'VERB'), ('Email', 'NOUN'), ('.', '.')],
              [('Zoom', 'VERB'), ('in','ADP'), ('.', '.')], [('zoom', 'VERB'), ('in','ADP'), ('.', '.')],
              [('Restart', 'VERB'), ('the','ADP'), ('application','NOUN'),  ('.', '.')],
              [('restart', 'VERB'), ('the','ADP'), ('application','NOUN'),  ('.', '.')],
              [('reboot', 'VERB'), ('the','ADP'), ('system', 'NOUN'), ('.', '.')],
              [('Reboot', 'VERB'), ('the','ADP'), ('laptop','NOUN'),  ('.', '.')]]

In [4]:
size = int(len(brown_tagged_sents) * 0.9)
size

51606

In [5]:
patterns = [
     (r'^open|^close|^shutdown|^Open|^Close|^Shutdown', 'VERB'),
     (r'.*ing$', 'VERB'),               # gerunds
     (r'.*in$', 'VERB'), 
     (r'.*ed$', 'VERB'),                # simple past
     (r'.*es$', 'VERB'),                # 3rd singular present
     (r'.*ould$', 'MD'),               # modals
     (r'.*\'s$', 'NOUN'),               # possessive nouns
     (r'.*s$', 'NOUN'),                 # plural nouns
     (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
                          # nouns (default)
 ]


In [6]:
%%time
default_tagger = nltk.DefaultTagger('NOUN')
regexp_tagger= nltk.RegexpTagger(patterns, backoff = default_tagger)
train_sents = brown_tagged_sents[:size] 
test_sents = brown_tagged_sents[size:]
train_sents = train_sents + extra_sents

Wall time: 1 ms


In [7]:
%%time
unigram_tagger = nltk.UnigramTagger(train_sents, backoff = regexp_tagger)
bigram_tagger = nltk.BigramTagger(train_sents, backoff = unigram_tagger)
t3 = nltk.TrigramTagger(train_sents, backoff = bigram_tagger)


Wall time: 1min 2s


In [8]:
t3.evaluate(test_sents)

0.9487437396534021

In [22]:
sent = 'Close all the open tabs'
tokens = nltk.word_tokenize(sent)
nltk.pos_tag(tokens, tagset = 'universal')

[('Close', 'ADV'),
 ('all', 'DET'),
 ('the', 'DET'),
 ('open', 'ADJ'),
 ('tabs', 'NOUN')]

In [30]:
t3.tag(tokens)

[('Close', 'VERB'),
 ('all', 'PRT'),
 ('the', 'DET'),
 ('open', 'ADJ'),
 ('tabs', 'NOUN')]

In [33]:
sent = 'Close Youtube'
tokens = nltk.word_tokenize(sent)
t3.tag(tokens)

[('Close', 'VERB'), ('Youtube', 'NOUN')]

In [34]:
sent = 'Open Youtube'
tokens = nltk.word_tokenize(sent)
t3.tag(tokens)

[('Open', 'VERB'), ('Youtube', 'NOUN')]

In [37]:
sent = 'Please close Youtube'
tokens = nltk.word_tokenize(sent)
t3.tag(tokens)

[('Please', 'VERB'), ('close', 'VERB'), ('Youtube', 'NOUN')]

In [5]:
freqDist = nltk.FreqDist(brown_words)
freq_keys = freqDist.keys()


In [172]:
def in_brown(word, keys):
    if word in list(keys):
        return True
    else:
        return False

In [18]:
def find_tag(word):
    return nltk.pos_tag([word], tagset = 'universal')

In [9]:
cond_freq = nltk.ConditionalFreqDist(tuples for lists in brown_tagged_sents+extra_sents for tuples in lists)

In [11]:
testing_words = ['close', 'open', 'Close', 'Open', 'minimize', 'shutdown', 'Shutdown', 'Minimize', 'satisfying', 'Login', 'zoom', 'Zoom', 'please', 'Please']
for word in testing_words:
    print(word +': {}'.format(dict(cond_freq[word])))

close: {'NOUN': 15, 'ADJ': 80, 'ADV': 96, 'VERB': 39}
open: {'ADJ': 239, 'ADV': 8, 'VERB': 55, 'NOUN': 4}
Close: {'ADV': 3, 'NOUN': 1, 'VERB': 5}
Open: {'NOUN': 8, 'ADJ': 4, 'VERB': 4}
minimize: {'VERB': 16}
shutdown: {'NOUN': 3, 'VERB': 3}
Shutdown: {'VERB': 3}
Minimize: {'VERB': 1}
satisfying: {'ADJ': 5, 'VERB': 8}
Login: {'VERB': 1}
zoom: {'VERB': 1}
Zoom: {'VERB': 1}
please: {'VERB': 37, 'PRT': 8}
Please: {'VERB': 10, 'PRT': 7}


In [16]:
sent = 'turn in my PC'
tokens = nltk.word_tokenize(sent)
t3.tag(tokens)

[('turn', 'VERB'), ('in', 'ADP'), ('my', 'DET'), ('PC', 'NOUN')]

In [ ]:
nltk.pos_tag(tokens, tagset = 'universal')

In [173]:
in_brown('Close', freq_keys)

True

In [174]:
freqDist['close']

230

In [178]:
find_tag('my')

[('my', 'PRON')]

In [31]:
from pickle import dump, load
output = open('tagger.pkl', 'wb')
dump(t3, output, -1)
output.close()